## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Import packages
import pandas as pd
import gzip as gz
import seaborn as sb

# Add extra relevant packages below as needed: 
import sqlite3 as sq
import shutil

In [ ]:
# Unzip IMDB SQL to new folder repository
import zipfile, os
with zipfile.ZipFile("zippedData/im.db.zip", 'r') as zip_ref:
    zip_ref.extractall("temp/IMDB_sql")
    
db_file = 'temp/IMDB_sql/im.db'

In [ ]:
# Initializing imdb SQL database
# Create a SQL connection to our SQLite database
con = sq.connect(db_file)


## Connection test and reference
test_table = pd.read_sql(""" SELECT *
                
                FROM movie_basics
                WHERE start_year = 2018

""", con)

test_table

## Data EDA and Cleanup

### Movie Budget EDA and Cleanup (Brian)

In [9]:
# Budget data load, eda and cleanup
tn_budget_raw = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
tn_budget_raw.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


### Movie Box Office Gross EDA and Cleanup (Elijah)

In [1]:
# Gross data load, eda and cleanup

### Movie Reviews EDA and Cleanup (Timi)

In [ ]:
# Review data load, eda and cleanup

#### Notebook Cleanup

In [7]:
#remove temporary files
con.close()
shutil.rmtree('temp/')